<a href="https://colab.research.google.com/github/GoogTech/langchain-tutorials/blob/master/Agents/Build_an_Agent.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Refer to : https://python.langchain.com/docs/tutorials/agents/

# Overview

By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. **Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs necessary to perform the action.**

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

# Installation

In [3]:
!pip install -U langchain langgraph langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [2]:
!pip install -U tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.7 MB/s eta 0:00:00


In [45]:
!pip show langchain langgraph langchain_openai

Name: langchain
Version: 0.3.18
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community
---
Name: langgraph
Version: 0.2.71
Summary: Building stateful, multi-actor applications with LLMs
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-sdk
Required-by: 
---
Name: langchain-openai
Version: 0.3.5
Summary: An integration package connecting OpenAI and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, openai, tiktoken
Required-by: 


# Define tools

We first need to create the tools we want to use. Our main tool of choice will be Tavily - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.

In [12]:
import os
from google.colab import userdata

os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [13]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1739362843, 'localtime': '2025-02-12 04:20'}, 'current': {'last_updated_epoch': 1739362500, 'last_updated': '2025-02-12 04:15', 'temp_c': 6.7, 'temp_f': 44.1, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 8.9, 'wind_kph': 14.4, 'wind_degree': 77, 'wind_dir': 'ENE', 'pressure_mb': 1013.0, 'pressure_in': 29.92, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 89, 'cloud': 100, 'feelslike_c': 3.9, 'feelslike_f': 39.1, 'windchill_c': 4.1, 'windchill_f': 39.4, 'heatindex_c': 6.3, 'heatindex_f': 43.4, 'dewpoint_c': 4.9, 'dewpoint_f': 40.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 12.4, 'gust_kph': 19.9}}"}, {'url': 'https://weathershogun.com/weather/usa/ca/

# Using Language Models

Next, let's learn how to use a language model to call tools. LangChain supports many different language models that you can use interchangably.

In [17]:
import os
from google.colab import userdata
from langchain_core.messages import HumanMessage
from langchain.chat_models import init_chat_model

# Initialize a ChatModel from the model name and provider
model = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    openai_api_key=userdata.get('OPENAI_API_KEY'),
    base_url=userdata.get('BASE_URL'),
)

# Call the model
response = model.invoke([HumanMessage(content="hi!")])

# Print the response from model
print(response, '\n')
print(response.content)

content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f3927aa00d', 'finish_reason': 'stop', 'logprobs': None} id='run-15834023-4871-461a-9d47-e6922a861e52-0' usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}} 

Hello! How can I assist you today?


We can now see what it is like to enable this model to do tool calling. In order to enable that we use `.bind_tools` to give the language model knowledge of these tools

In [19]:
import os
from google.colab import userdata
from langchain_core.messages import HumanMessage
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults

# Initialize the search tool
search = TavilySearchResults(max_results=2)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

# Initialize a ChatModel from the model name and provider
model = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    openai_api_key=userdata.get('OPENAI_API_KEY'),
    base_url=userdata.get('BASE_URL'),
)

# Call the model
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

# Print the response from model
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


Now, let's try calling it with some input that would expect a tool to be called.

In [20]:
import os
from google.colab import userdata
from langchain_core.messages import HumanMessage
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults

# Initialize the search tool
search = TavilySearchResults(max_results=2)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

# Initialize a ChatModel from the model name and provider
model = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    openai_api_key=userdata.get('OPENAI_API_KEY'),
    base_url=userdata.get('BASE_URL'),
)

# Call the model
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

# Print the response from model
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco current weather'}, 'id': 'call_qNdbkuX8y1stCkc0sfuwF5qe', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

**This isn't calling that tool yet - it's just telling us to.** In order to actually call it, we'll want to create our agent.

# Create the agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using LangGraph to construct the agent.

In [21]:
import os
from google.colab import userdata
from langchain_core.messages import HumanMessage
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent

# Initialize the search tool
search = TavilySearchResults(max_results=2)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

# Initialize a ChatModel from the model name and provider
model = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    openai_api_key=userdata.get('OPENAI_API_KEY'),
    base_url=userdata.get('BASE_URL'),
)

# Initialize the agent with the LLM and the tools
agent_executor = create_react_agent(model, tools)

# Run the agent

First up, let's see how it responds when there's no need to call a tool:

In [22]:
import os
from google.colab import userdata
from langchain_core.messages import HumanMessage
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent

# Initialize the search tool
search = TavilySearchResults(max_results=2)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

# Initialize a ChatModel from the model name and provider
model = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    openai_api_key=userdata.get('OPENAI_API_KEY'),
    base_url=userdata.get('BASE_URL'),
)

# Initialize the agent with the LLM and the tools
agent_executor = create_react_agent(model, tools)

# Run the agent
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})
response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='7913c79a-0753-4f6e-9c15-000c6f0ce0d8'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f3927aa00d', 'finish_reason': 'stop', 'logprobs': None}, id='run-48a470af-cbf0-44a9-9357-6df71c47b461-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

Let's now try it out on an example where it should be invoking the tool:

In [27]:
import os
from google.colab import userdata
from langchain_core.messages import HumanMessage
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent

# Initialize the search tool
search = TavilySearchResults(max_results=2)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

# Initialize a ChatModel from the model name and provider
model = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    openai_api_key=userdata.get('OPENAI_API_KEY'),
    base_url=userdata.get('BASE_URL'),
)

# Initialize the agent with the LLM and the tools
agent_executor = create_react_agent(model, tools)

# Run the agent
response = agent_executor.invoke({"messages": [HumanMessage(content="whats the weather in sf?")]})
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='e6e2f06b-8572-493c-8783-008fc743760a'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FAJJiTFIWnxb2ATfghA03e6H', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f3927aa00d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cd2df1f5-c724-4201-8399-983ba60132d3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_FA

# Streaming Messages

We've seen how the agent can be called with `.invoke` to get a final response. If the agent executes multiple steps, this may take a while. To show intermediate progress, we can stream back messages as they occur.

In [28]:
import os
from google.colab import userdata
from langchain_core.messages import HumanMessage
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent

# Initialize the search tool
search = TavilySearchResults(max_results=2)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

# Initialize a ChatModel from the model name and provider
model = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    openai_api_key=userdata.get('OPENAI_API_KEY'),
    base_url=userdata.get('BASE_URL'),
)

# Initialize the agent with the LLM and the tools
agent_executor = create_react_agent(model, tools)

# Run the agent
for chunk in agent_executor.stream({"messages": [HumanMessage(content="whats the weather in sf?")]}):
    print(chunk)
    print('---')

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_orgSD0AQpT7RCreOkoJRtnzs', 'function': {'arguments': '{"query":"San Francisco weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 86, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f3927aa00d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-71f5d8ce-c6b0-452c-a5b5-54e94f9c1351-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather today'}, 'id': 'call_orgSD0AQpT7RCreOkoJRtnzs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 21, 'total_tokens': 107,

# Streaming tokens❌
... It's confusing for beginners ...

# Adding in memory

As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a thread_id when invoking the agent (so it knows which thread/conversation to resume from).

In [35]:
import os
from google.colab import userdata
from langchain_core.messages import HumanMessage
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# Initialize the search tool
search = TavilySearchResults(max_results=2)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

# Initialize a ChatModel from the model name and provider
model = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    openai_api_key=userdata.get('OPENAI_API_KEY'),
    base_url=userdata.get('BASE_URL'),
)

# Initialize the in-memory checkpoint saver
memory = MemorySaver()

# Initialize the agent with the LLM, tools and the memory
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [41]:
# This enables us to support multiple conversation threads with a single application,
# a common requirement when your application has multiple users.
config = {"configurable": {"thread_id": "thread_id_000"}}

# Run the agent
for chunk in agent_executor.stream(
    input={"messages": [HumanMessage(content="Hi I'm Hack")]},
    config=config,
):
    print(chunk)

{'agent': {'messages': [AIMessage(content="Hello, Hack! It's nice to see you again. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 176, 'total_tokens': 195, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f3927aa00d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bfef2a7e-eb26-4f35-b4ad-be7be069d841-0', usage_metadata={'input_tokens': 176, 'output_tokens': 19, 'total_tokens': 195, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}


In [42]:
# This enables us to support multiple conversation threads with a single application,
# a common requirement when your application has multiple users.
config = {"configurable": {"thread_id": "thread_id_000"}}

# Run the agent
for chunk in agent_executor.stream(
    input={"messages": [HumanMessage(content="What's my name?")]},
    config=config,
):
    print(chunk)

{'agent': {'messages': [AIMessage(content='Your name is Hack. How can I assist you further?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 206, 'total_tokens': 219, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f3927aa00d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1d3c8070-42a5-4c66-a756-8215c03ec716-0', usage_metadata={'input_tokens': 206, 'output_tokens': 13, 'total_tokens': 219, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}


If you want to start a new conversation, all you have to do is change the thread_id used:

In [43]:
# This enables us to support multiple conversation threads with a single application,
# a common requirement when your application has multiple users.
config = {"configurable": {"thread_id": "thread_id_111"}}

# Run the agent
for chunk in agent_executor.stream(
    input={"messages": [HumanMessage(content="What's my name?")]},
    config=config,
):
    print(chunk)

{'agent': {'messages': [AIMessage(content='I still don’t have that information. If you let me know your name, I can remember it for the duration of our conversation!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 128, 'total_tokens': 156, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f3927aa00d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1772330c-0e8e-4f1f-8078-b22138f47234-0', usage_metadata={'input_tokens': 128, 'output_tokens': 28, 'total_tokens': 156, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
